In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Masking, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# --- Load and Clean Data ---
df = pd.read_csv("number_sequence_distraction_data.csv")
df = df[df["clicked_sequence"].notnull()]  

# --- Convert clicked_sequence to list of integers ---
sequences = df["clicked_sequence"].apply(lambda x: list(map(int, str(x).split())))
max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_len, padding='post')
X = X / 9.0  
X = X.reshape((X.shape[0], X.shape[1], 1))  

# --- Normalize distraction_level ---
scaler = MinMaxScaler()
y = scaler.fit_transform(df[["distraction_level"]].values).flatten()

# --- Custom Accuracy (±0.2 tolerance) ---
def custom_accuracy(y_true, y_pred):
    return K.mean(K.less_equal(K.abs(y_true - y_pred), 0.2))

# --- Split Data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- GRU Model ---
model = Sequential([
    Masking(mask_value=0.0, input_shape=(max_len, 1)),
    GRU(64, return_sequences=True),
    Dropout(0.3),
    GRU(32),
    Dense(32, activation='relu'),
    Dense(1)  
])

model.compile(optimizer='adam', loss='mse', metrics=['mae', custom_accuracy])

# --- Train the Model ---
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=8,
                    validation_data=(X_test, y_test), callbacks=[early_stop], verbose=1)

# --- Evaluate ---
loss, mae, acc = model.evaluate(X_test, y_test)
print(f"\n✅ Final Test MAE: {mae:.4f}")
print(f"🎯 Tolerance Accuracy (±0.2): {acc*100:.2f}%")

# --- Diagnostic: Predictions vs Actual ---
y_pred = model.predict(X_test).flatten()
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

for true, pred in zip(y_test_rescaled[:10], y_pred_rescaled[:10]):
    print(f"True: {true:.2f}, Predicted: {pred:.2f}")


Epoch 1/100
20/20 [==============================] - 7s 96ms/step - loss: 0.0977 - mae: 0.2371 - custom_accuracy: 0.5437 - val_loss: 0.0690 - val_mae: 0.2178 - val_custom_accuracy: 0.4250
Epoch 2/100
20/20 [==============================] - 0s 12ms/step - loss: 0.0861 - mae: 0.2262 - custom_accuracy: 0.5437 - val_loss: 0.0623 - val_mae: 0.2026 - val_custom_accuracy: 0.5500
Epoch 3/100
20/20 [==============================] - 0s 8ms/step - loss: 0.0843 - mae: 0.2269 - custom_accuracy: 0.5375 - val_loss: 0.0570 - val_mae: 0.1920 - val_custom_accuracy: 0.6000
Epoch 4/100
20/20 [==============================] - 0s 8ms/step - loss: 0.0810 - mae: 0.2231 - custom_accuracy: 0.5500 - val_loss: 0.0577 - val_mae: 0.1943 - val_custom_accuracy: 0.5750
Epoch 5/100
20/20 [==============================] - 0s 8ms/step - loss: 0.0779 - mae: 0.2221 - custom_accuracy: 0.5625 - val_loss: 0.0547 - val_mae: 0.1892 - val_custom_accuracy: 0.6250
Epoch 6/100
20/20 [==============================] - 0s 8ms/ste